In [ ]:
import time
import glob
from pathlib import Path
import sys
import s3fs
from typing import List, Optional, Dict
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import pyarrow.parquet as pq
import fasttext
import os
import warnings
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger
from tensorboard.backend.event_processing import event_accumulator
import torch
from torch import nn
from torch.optim import Adam, SGD
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
import unidecode
# from src.model_negsamp import FastTextModule_negsamp, FastTextModel_negsamp
from src.model import FastTextModule, FastTextModel
from src.dataset import FastTextModelDataset
from src.tokenizer import NGramTokenizer
from src.preprocess import clean_text_feature
import warnings
import nltk
nltk.download('stopwords')
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt

In [ ]:
embedding_dim = 20
vocab_size = 100
num_classes=  35
categorical_vocabulary_sizes = [5,7]
padding_idx = 0
sparse = True

embeddings = nn.Embedding(
    embedding_dim=embedding_dim,
    num_embeddings=vocab_size,
    padding_idx=padding_idx,
    sparse=sparse,
)

categorical_embeddings = {}

for var_idx, vocab_size in enumerate(categorical_vocabulary_sizes):
    emb = nn.Embedding(embedding_dim=embedding_dim, num_embeddings=vocab_size)
    categorical_embeddings[var_idx] = emb
    setattr(self, "emb_{}".format(var_idx), emb)

self.fc = nn.Linear(embedding_dim, num_classes)

    def forward(self, inputs: List[torch.LongTensor]) -> torch.Tensor:
        """
        Forward method.

        Args:
            inputs (List[torch.LongTensor]): Model inputs.

        Returns:
            torch.Tensor: Model output.
        """
        # Embed tokens
        x_1 = inputs[0]
        x_1 = self.embeddings(x_1)

        x_cat = []
        for i, (variable, embedding_layer) in enumerate(
            self.categorical_embeddings.items()
        ):
            x_cat.append(embedding_layer(inputs[i + 1]))

        # Mean of tokens
        non_zero_tokens = x_1.sum(-1) != 0
        non_zero_tokens = non_zero_tokens.sum(-1)
        x_1 = x_1.sum(dim=-2)
        x_1 /= non_zero_tokens.unsqueeze(-1)
        x_1 = torch.nan_to_num(x_1)

        if x_cat != []:
            x_in = x_1 + torch.stack(x_cat, dim=0).sum(dim=0)
        else:
            x_in = x_1

        # Linear layer
        z = self.fc(x_in)
        return z
